In [1]:
import networkx as nx
import metis
import os
import numpy as np

In [10]:
G=nx.Graph()
f=open("roadNet-CA.txt","r")
f.readline()
f.readline()
b=f.readline()
f.readline()
a=f.readline()
while(a):
    G.add_edge(int(a.split()[0])+1,int(a.split()[1])+1)
    a=f.readline()

In [ ]:
#C=metis.part_graph(G,4)

In [12]:
d=G.edge
f=open("roadNet-CA-Graclus.txt","w")
f.write("1965206 2766607")
for i in range(1+max(d.keys())):
    f.write('\n')
    try:
        for j in d[i].keys():
            f.write(str(j)+' ')
    except KeyError:
        f.write('\n')
f.close()

In [13]:
os.system("gpmetis roadNet-CA-Graclus.txt 4") #Partitioning the graph into 4 clusters

NameError: name 'os' is not defined

In [ ]:
part = f.open("roadNet-CA-Graclus.txt.part.4").read().split()
parts=[]
for i in part:
    parts.append(int(i))

In [2]:
def recursiveFormOmega(G,fileName,c,k):
    d=G.edge
    f=open(fileName,"w")
    f.write(str(len(G.node))+" "+str(len(G.edges())))
    for i in range(1+max(d.keys())):
        f.write('\n')
        try:
            for j in d[i].keys():
                f.write(str(j)+' ')
        except KeyError:
            f.write('\n')
    f.close()
    os.system("gpmetis "+fileName+" "+str(c))
    part = f.open(fileName+".part."+str(c)).read().split()
    parts=[]
    for i in part:
        parts.append(int(i))
    clusters=[]
    for j in range(c):
        temp=[]
        for i in range(len(parts)):
            if parts[i]==j:
                temp.append(i+1)
        clusters.append(temp)
    eVal=[]
    eVec=[]
    for i in range(c):
        l=len(clusters[i])
        Ai=[]
        try:
            for j in range(l):
                temp=[]
                for m in range(l):
                    if G.has_edge(clusters[i][j],clusters[i][m]):
                        temp.append(1)
                    else:
                        temp.append(0)
                Ai.append(temp)
            val,vec=np.linalg.eig(Ai)    
        except MemoryError:
            pass
        eVal=eVal+list(val)
        eVec=eVec+list(vec.T)
    eigen=zip(eVal,eVec)
    eigen.sort()
    finalEigen=eigen[:k]
    return [list(zip(*finalEigen)[0]),list(zip(*finalEigen)[1])]
    #eigenVectors=zip(*eigen)[1]
    #omega=[]
    #for v in eigenVectors:
    #    v=v+([0]*(d-len(v)))
    #    omega.append(v)
    #omega=np.array(omega)
    #omega=omega.T
    #return omega

In [11]:
def blockLanczoz(A,B0,Q0.Q1):
    B=[B0]
    Q=[Q0,Q1]
    T=[[]]
    Qj=[Q1]
    for j in range(1,50):
        R=np.dot(A,Q[j])-np.dot(Q[j-1],B[j-1].T)
        Aj=np.dot(Q[j].T,R)
        R=R-np.dot(Q[j],Aj)
        B.append([])
        Q.append([])
        T.append([])
        Q[j+1],B[j]=np.linalg.qr(R)
        T[j]=np.dot(np.dot(Qj.T,A),Qj)
        Qj.append(Q[j+1])